In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import pymysql
      
#id
def get_id():
    id_n = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for id_c in soup.find_all('a','card-link'):
        #print(id_c.get('href'))
        n = id_c.get('href')
        newid = n[23:]
        #print(newid)
        id_n.append(newid)
    return id_n

#地點名稱       
def get_site():
    site = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for ele in soup.select('div.card-title'):
        #print(ele.text)
        site.append(ele.text)
    return site

def get_Info(s):
    nattr = []
    for j in range(1,len(s)+1):
        num = str(j)
        
        page = driver.find_element_by_xpath("/html/body/form/div[5]/div[3]/div[3]/ul/li[" + num + "]/div/div/a")
        #解決elenium中無法點擊Element：ElementClickInterceptedException
        driver.execute_script("arguments[0].click();", page)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #n = str(i[j-1])
        action = soup.find('form', id='form1').get('action')
        re_a = action.replace('./','https://www.taiwan.net.tw/')

        driver.get(re_a)
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')

        #sitename & attr
        cn = 0
        col = []
        for q in soup2.select('div.hashtag a'):            
            #attr
            att = q.text
            newatt = att[1:]
            col.append(newatt)
        nattr.append(col)
        
        driver.back()
    return nattr

In [2]:
def get_url():
    curhref = db.cursor()

    target = []
    findhref = "SELECT * FROM site_href"
    curhref.execute(findhref)
    for h in curhref.fetchall():
        t = []
        city = h[1]
        href = h[2]
        t.append(city)
        t.append(href)
        target.append(t)
    return target

In [3]:
def getSites(city_name, url):
    driver.get(url)
    i = get_id()
    s = get_site()
    tag = get_Info(s)
    
    cursor = db.cursor()
    cursor2 = db.cursor()
    
    from_id = []
    to_id = []
    f = []
    to = []

    for m in range(len(s)): #42
        name = s[m]
        
        qurey = "SELECT * FROM site_data WHERE name = '" + name + "' AND city_name ='" + city_name + "'"
        test = cursor.execute(qurey)
        fet = cursor.fetchall()
        if not fet:
            #資料表中沒有的先不管
            f.append("NONE")
        else:
            for row in fet:
                fin = row[0]
                f.append(fin)
                #print(fin)

        if(f[m] != 'NONE'):
            for j in range(len(tag[m])):
                # print(tag[i])
                # from_id
                attr = tag[m][j]
                from_id.append(f[m])

                qureyto = "SELECT * FROM site_attr WHERE type = '' AND tag = '" + attr +"'"
                cursor2.execute(qureyto)
                for row2 in cursor2.fetchall():
                    # to_id
                    tin = row2[0]
                    to_id.append(tin)  

    add_Relationship(from_id, to_id)
    # print(from_id)
    # print(len(from_id)) #43
    # print(to_id)
    # print(len(to_id)) #43

In [4]:
def add_Relationship(from_id, to_id):
    cux = db.cursor()
    for each in range(len(from_id)):
        add_data = ("INSERT INTO site_relationship"
                    "(from_id, to_id) "
                    "VALUES (%s,%s)")
        data = (from_id[each], to_id[each])
        cux.execute(add_data, data)
#         print(data)
        db.commit()

In [5]:
#main
tStart = time.time()
driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")

db = pymysql.connect(
    host="localhost",
    user="root",
    passwd="12345",
    database='homestead'
)

url = get_url()
# url = "https://www.taiwan.net.tw/m1.aspx?sNo=0001123"

for k in range(len(url)):
    getSites(url[k][0], url[k][1])
# getSites('台東', url)

tEnd = time.time()
print("Run Time：", tEnd - tStart)

Run Time： 3186.745805978775


In [17]:
# s = get_site()
# tag = get_Info(s)

# print(s)